In [2]:
# import libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import types
from botocore.client import Config
import ibm_boto3 # library to import the csv file

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors # Matplotlib and associated plotting modules

from sklearn import linear_model # import the linear model

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    cer

In [3]:
def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_af763645a80e4581b33d7a515afea6a2 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='XrxDieoZyhzmuQppad4HmidYSGfGhjisHoMUBYwI0vD1',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_af763645a80e4581b33d7a515afea6a2.get_object(Bucket='testproject-donotdelete-pr-pprugu4vcembmg',Key='Rating for Places.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

original_rating = pd.read_csv(body)
original_rating.head()

,Places,Rating
0,"Baltimore, the United States",6
1,"Beijing, China",3
2,"Boston, the United States",5
3,"Cambridge, the United Kingdom",8
4,"Chengdu, China",5


In [4]:
# append the data latitude and longitude of places to the list
latitude_list = list()
longitude_list = list()
for i in range (0, len(original_rating)):
    address = original_rating['Places'][i]
    latitude = Nominatim(user_agent="foursquare_agent").geocode(address).latitude
    longitude = Nominatim(user_agent="foursquare_agent").geocode(address).longitude
    latitude_list.append(latitude)
    longitude_list.append(longitude)
    #print('The geograpical coordinate of {} are {}, {}.'.format(original_rating['Places'][i], latitude, longitude))
    
df = pd.DataFrame(original_rating) 
new_data = df.assign(Latitude = latitude_list, Longitude = longitude_list) 
print(new_data.shape)
new_data.head()

(30, 4)


,Places,Rating,Latitude,Longitude
0,"Baltimore, the United States",6,39.290882,-76.610759
1,"Beijing, China",3,39.906217,116.391276
2,"Boston, the United States",5,42.360253,-71.058291
3,"Cambridge, the United Kingdom",8,52.203482,0.123582
4,"Chengdu, China",5,30.662420,104.063322


In [5]:
# create map using latitude and longitude values
map_rating = folium.Map(location=[latitude, longitude], zoom_start=3)
# add markers to map
for lat, lng, label in zip(new_data['Latitude'], new_data['Longitude'], new_data['Places']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_rating)  
map_rating

In [6]:
# Define Foursquare Credentials and Version
CLIENT_ID = 'BCIYDZYRYJJ4LJCUQLMSAEHRC1TPNOSFD2U1ANFMM5XK0MQQ' # your Foursquare ID
CLIENT_SECRET = 'HXMVCGI4YO0ULUWC30XJOO3O01JKNERPGHBJSQ2ERC25YVF5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BCIYDZYRYJJ4LJCUQLMSAEHRC1TPNOSFD2U1ANFMM5XK0MQQ
CLIENT_SECRET:HXMVCGI4YO0ULUWC30XJOO3O01JKNERPGHBJSQ2ERC25YVF5


In [7]:
# Create a function to get all the neighborhoods in places
def getNearbyVenues(names, latitudes, longitudes, radius=10000):
    LIMIT = 100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Place Name', 
                  'Place Latitude', 
                  'Place Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [8]:
places_venues = getNearbyVenues(names=new_data['Places'],
                                   latitudes=new_data['Latitude'],
                                   longitudes=new_data['Longitude']
                                  )

Baltimore, the United States
Beijing, China
Boston, the United States
Cambridge, the United Kingdom
Chengdu, China
Chicago, the United States
Chongqing, China
Innsbruck, Austria
Jackson Hole, the United States
Jerusalem, Israel
Katmandu, Nepal
Kyoto, Japan
London, the United Kingdom
Los Angeles, the United States
Miami, the United States
Munich, Germany
Nagoya, Japan
Nara, Japan
New York, the United States
Osaka, Japan
Philadelphia, the United States
Guilin, China
Pokhara, Nepal
Redding, the United States
Salzburg, Austria
Shanghai, China
Vienna, Austria
Washington D,C., the United States
Weifang, China
Wuxi, China


In [9]:
print(places_venues.shape)
places_venues.head()

(2556, 7)


,Place Name,Place Latitude,Place Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Baltimore, the United States",39.290882,-76.610759,Baltimore Farmers' Market & Bazaar,39.293297,-76.610490,Farmers Market
1,"Baltimore, the United States",39.290882,-76.610759,Sotto Sopra,39.294314,-76.615375,Italian Restaurant
2,"Baltimore, the United States",39.290882,-76.610759,Center Club,39.287194,-76.613959,Lounge
3,"Baltimore, the United States",39.290882,-76.610759,Barnes & Noble,39.286142,-76.607857,Bookstore
4,"Baltimore, the United States",39.290882,-76.610759,Inner Harbor,39.286590,-76.610704,Neighborhood


In [10]:
# check how many venues were returned for each neighborhood
places_venues.groupby('Place Name').count()

,Place Latitude,Place Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Place Name,,,,,,
"Baltimore, the United States",100,100,100,100,100,100
"Beijing, China",100,100,100,100,100,100
"Boston, the United States",100,100,100,100,100,100
"Cambridge, the United Kingdom",100,100,100,100,100,100
"Chengdu, China",100,100,100,100,100,100
"Chicago, the United States",100,100,100,100,100,100
"Chongqing, China",82,82,82,82,82,82
"Guilin, China",36,36,36,36,36,36
"Innsbruck, Austria",100,100,100,100,100,100


In [12]:
# count the unique venue in the data
print('There are {} uniques categories.'.format(len(places_venues['Venue Category'].unique())))

There are 318 uniques categories.


In [13]:
# one hot encoding
places_onehot = pd.get_dummies(places_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
places_onehot['Place Name'] = places_venues['Place Name'] 

# move neighborhood column to the first column
fixed_columns = [places_onehot.columns[-1]] + list(places_onehot.columns[:-1])
places_onehot = places_onehot[fixed_columns]

print(places_onehot.shape)
places_onehot.head()

(2556, 319)


,Place Name,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Amphitheater,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Austrian Restaurant,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Beijing Restaurant,Belgian Restaurant,Bike Shop,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Buddhist Temple,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Café,Cajun / Creole Restaurant,Campground,Canal,Candy Store,Cantonese Restaurant,Casino,Castle,Cave,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,City,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish Market,Flower Shop,Food,Food Court,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Inn,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jiangsu Restaurant,Juice Bar,Korean Restaurant,Kushikatsu Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Manti Place,Market,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Nabe Restaurant,Nail Salon,National Park,Neighborhood,New American Restaurant,Nightclub,Noodle House,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Palace,Park,Parking,Pastry Shop,Pedestrian Plaza,Peking Duck Restaurant,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Planetarium,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Public Art,RV Park,Ramen Restaurant,Record Shop,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Road,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Shrine,Ski Area,Smoke Shop,Snack Place,Soba Restaurant,Soccer Field,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,State / Provincial Park,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaur

In [14]:
# get the data for grouping 
place_grouped = places_onehot.groupby('Place Name').mean().reset_index()
place_grouped = place_grouped.merge(original_rating, left_on='Place Name', right_on='Places')
place_grouped = place_grouped.drop(['Places'], axis=1)
print(place_grouped.shape)
place_grouped.head()

(30, 320)


,Place Name,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Amphitheater,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Austrian Restaurant,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Beijing Restaurant,Belgian Restaurant,Bike Shop,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Buddhist Temple,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Café,Cajun / Creole Restaurant,Campground,Canal,Candy Store,Cantonese Restaurant,Casino,Castle,Cave,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,City,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish Market,Flower Shop,Food,Food Court,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Inn,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jiangsu Restaurant,Juice Bar,Korean Restaurant,Kushikatsu Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Manti Place,Market,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Nabe Restaurant,Nail Salon,National Park,Neighborhood,New American Restaurant,Nightclub,Noodle House,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Palace,Park,Parking,Pastry Shop,Pedestrian Plaza,Peking Duck Restaurant,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Planetarium,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Public Art,RV Park,Ramen Restaurant,Record Shop,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Road,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Shrine,Ski Area,Smoke Shop,Snack Place,Soba Restaurant,Soccer Field,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,State / Provincial Park,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaur

In [15]:
num_top_venues = 3

for hood in place_grouped['Place Name']:
    print("----"+hood+"----")
    temp = place_grouped[place_grouped['Place Name'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.drop(319, axis = 0)
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Baltimore, the United States----
                venue  freq
0  Italian Restaurant  0.05
1         Coffee Shop  0.05
2  Seafood Restaurant  0.04


----Beijing, China----
           venue  freq
0  Historic Site  0.18
1          Hotel  0.15
2           Park  0.07


----Boston, the United States----
    venue  freq
0    Park  0.08
1  Bakery  0.08
2     Gym  0.04


----Cambridge, the United Kingdom----
          venue  freq
0           Pub  0.13
1          Café  0.10
2  Burger Joint  0.06


----Chengdu, China----
           venue  freq
0    Coffee Shop  0.16
1          Hotel  0.14
2  Shopping Mall  0.08


----Chicago, the United States----
     venue  freq
0    Hotel  0.10
1     Park  0.07
2  Theater  0.05


----Chongqing, China----
                  venue  freq
0                 Hotel  0.18
1           Coffee Shop  0.17
2  Fast Food Restaurant  0.10


----Guilin, China----
                  venue  freq
0                 Hotel  0.19
1                Hostel  0.11
2  Fast Food Restaurant

In [17]:
# get the place for prediction
place_test = input('Please the place you want to predict:')  # Saint Petersburg, Russia
test_data = [[place_test]] 
test_original_data = pd.DataFrame(test_data, columns = ['Place']) 

# append the data latitude and longitude of places to the list
latitude_list = list()
longitude_list = list()
for i in range (0, len(test_original_data)):
    address = test_original_data['Place'][i]
    latitude = Nominatim(user_agent="foursquare_agent").geocode(address).latitude
    longitude = Nominatim(user_agent="foursquare_agent").geocode(address).longitude
    latitude_list.append(latitude)
    longitude_list.append(longitude)
    print('The geograpical coordinate of {} are {}, {}.'.format(test_original_data['Place'][i], latitude, longitude))

# get the normalized prediction place data based on its own creteria 
df = pd.DataFrame(test_original_data) 
test_new_data = df.assign(Latitude = latitude_list, Longitude = longitude_list) 

new_place_venues = getNearbyVenues(names=test_new_data['Place'],
                                   latitudes=test_new_data['Latitude'],
                                   longitudes=test_new_data['Longitude']
                                  )

new_place_onehot = pd.get_dummies(new_place_venues[['Venue Category']], prefix="", prefix_sep="") # one hot encoding
new_place_onehot['Place Name'] = new_place_venues['Place Name'] # add neighborhood column back to dataframe
fixed_columns = [new_place_onehot.columns[-1]] + list(new_place_onehot.columns[:-1])
new_place_onehot = new_place_onehot[fixed_columns] # move neighborhood column to the first column
new_place_grouped = new_place_onehot.groupby('Place Name').mean().reset_index() # get the data for grouping 

# get the normalized prediction place data based on the past creteria 
column_name = list()
for col in place_grouped.columns: 
    column_name.append(col)
column_name.remove('Place Name')
new_place_norm = pd.DataFrame(test_data, columns = ['Place']) 
for i in range(0, len(column_name)):
    new_place_norm[column_name[i]] = 0

new_column_name = list()
for col in new_place_grouped.columns: 
    new_column_name.append(col)
new_column_name.remove('Place Name')
for i in range(0, len(new_column_name)):
    for j in range(0, len(column_name)):
        if new_column_name[i] == column_name[j]:          
            #print(new_column_name[i], column_name.index(column_name[j]))
            new_place_norm.iloc[0, new_place_norm.columns.get_loc(column_name[j])] = new_place_grouped[new_column_name[i]][0]
            break
new_place_norm.head()

Please the place you want to predict:Saint Petersburg, Russia
The geograpical coordinate of Saint Petersburg, Russia are 59.938732, 30.316229.
Saint Petersburg, Russia


,Place,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Amphitheater,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Austrian Restaurant,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Beijing Restaurant,Belgian Restaurant,Bike Shop,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Buddhist Temple,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Café,Cajun / Creole Restaurant,Campground,Canal,Candy Store,Cantonese Restaurant,Casino,Castle,Cave,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,City,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish Market,Flower Shop,Food,Food Court,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Inn,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jiangsu Restaurant,Juice Bar,Korean Restaurant,Kushikatsu Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Manti Place,Market,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Nabe Restaurant,Nail Salon,National Park,Neighborhood,New American Restaurant,Nightclub,Noodle House,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Palace,Park,Parking,Pastry Shop,Pedestrian Plaza,Peking Duck Restaurant,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Planetarium,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Public Art,RV Park,Ramen Restaurant,Record Shop,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Road,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Shrine,Ski Area,Smoke Shop,Snack Place,Soba Restaurant,Soccer Field,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,State / Provincial Park,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,T

In [18]:
regr = linear_model.LinearRegression()
x = np.asanyarray(place_grouped[column_name[0:len(column_name)-1]])
y = np.asanyarray(place_grouped[['Rating']])
regr.fit (x, y)

# The coefficients
print ('Coefficients: ', regr.coef_)
print ('Intercept: ',regr.intercept_)

Coefficients:  [[-2.79511776e-01  4.08092422e-01 -2.17331537e+00 -1.05398330e+00
  -1.02764509e+00  2.09609339e+00 -8.21145833e-01 -3.63874781e+00
   1.28309069e-01  4.59727458e+00 -3.66480326e+00  1.25051318e+00
  -5.93548973e+00  5.95983756e+00  1.78173899e+00 -1.81950970e+00
   5.20502949e-01 -2.76687220e+00 -4.57727383e-01  5.46482724e+00
   1.18592388e+00  9.55820756e-01  2.10902879e+00  6.73212131e-01
  -8.26254088e-01 -8.17526253e-02 -7.48391574e-01  2.86315707e+00
  -2.10688360e+00  6.26965082e-01 -2.34380256e+00 -1.00459347e+00
  -5.60542036e-01  8.78289391e-01  1.07698713e+00 -1.43748049e+00
   4.41419215e+00  1.81172850e-01 -1.08505154e+00 -1.17639726e+00
  -3.18758677e+00  6.53813481e+00  6.26965082e-01 -3.16681656e+00
  -9.04487915e-01  2.53101940e+00  6.93441374e+00  5.18958850e+00
  -1.89010631e+00 -9.94454186e-01  2.72753540e+00  1.54937225e+00
  -7.54814781e-01 -1.05143533e-01  1.38134517e+00 -2.81089794e-01
  -9.94454186e-01 -6.27565445e-01  1.21035069e+00  4.98340723

In [19]:
# The prediction of the test place
test_prediction= regr.predict(new_place_norm[column_name[0:len(column_name)-1]])
print('The predicted rate of {} will be {}.'.format(place_test, round(test_prediction[0][0],3)))

The predicted rate of Saint Petersburg, Russia will be 5.822.
